In [1]:
#limpio la memoria
rm(list=ls())
gc()

#cargo librerias
library("data.table")
library("ROCR")
library("rpart")
library("rpart.plot")
library("caret")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,492161,26.3,1085284,58,641780,34.3
Vcells,941099,7.2,8388608,64,1753993,13.4


Warning message:
“package ‘ROCR’ was built under R version 3.6.3”
Warning message:
“package ‘rpart’ was built under R version 3.6.3”
Loading required package: lattice

Warning message:
“package ‘lattice’ was built under R version 3.6.3”
Loading required package: ggplot2



In [14]:

#Parametros entrada

#kcarpeta_datasets    <- "../input/laboratorio-de-implementacion-i-2021/"   #KAGGLE
kcarpeta_datasets    <- "~/buckets/b1/crudo/datasetsOri/"                          #VM o Ubuntu

#Archivo con datos etiquetados para entrenamiento
karchivo_entrada      <-  paste0(kcarpeta_datasets, "paquete_premium_202011.csv")

#Archivo con datos sin etiquetar para generar la predicción
karchivo_predecir      <-  paste0(kcarpeta_datasets, "paquete_premium_202101.csv")

#Formato para submit en Kaggle
#karchivo_score      <-  "../input/uamds2020ldi1f1/Sample_201910_Fase_I.txt"

#Separador de campos en archivos
kcampos_separador     <-  "\t"

#Campo que identifica las muestras
kcampo_id             <-  "numero_de_cliente"

#Campo que contiene la clase a estimar
kclase_nomcampo       <-  "clase_ternaria"

#Valor de interés
kclase_valor_positivo <-  "BAJA+2"

#Campos a borrar para el entrenamiento
kcampos_a_borrar      <-  c(kcampo_id,kclase_nomcampo,"foto_mes")

#Campo que contendrá a la variable objetivo generada
kobjetivo             <-  "clase"

#Identificación del modelo
kmodelo               <-  "02-RPART"

#Ganancia por TP
kTPGain               <-  48750

#Pérdida por FP
kFPGain               <-  -1250

kSeed <- 0


#Establezco semilla aleatoria
set.seed(kSeed)



In [15]:
#cargo los datos
dataset <- fread(karchivo_entrada)

#Para hacer pruebas rapidas puedo reducir el dataset a una fraccion
#subsample <- sample(1:nrow(dataset), .1 * nrow(dataset))
#dataset <- dataset[subsample,]

In [16]:
#Genero la clase
dataset[,(kobjetivo) := as.integer(dataset[,..kclase_nomcampo] == kclase_valor_positivo)]

#Limpio campos a borrar
dataset[ ,  (kcampos_a_borrar) := NULL    ] 

#Completo missings
dataset[is.na(dataset)] <- 0

Warning message in `[.data.table`(dataset, , ..kclase_nomcampo):
“Both 'kclase_nomcampo' and '..kclase_nomcampo' exist in calling scope. Please remove the '..kclase_nomcampo' variable in calling scope for clarity.”


In [17]:
# generacion del modelo
formula  <-  formula(paste(kobjetivo, "~ ."))

In [18]:
#Genero los 5 folds
folds <- createFolds(dataset$clase, 5, returnTrain = TRUE)

#Inicializo ganancia
ganancia_total <- 0


for (fold in c(1,2,3,4,5))
{

#Selecciono los registros train/test para cada fold
train <- dataset[unlist(folds[paste0('Fold',fold)]),]
test <- dataset[-unlist(folds[paste0('Fold',fold)]),]
    
#Entreno el modelo
modelo   <-  rpart(formula,   data = train, maxdepth = 5,  cp=0,   xval=0)   

#Aplico al set de test
predicted <- predict(modelo, test,) # predicted values

#Calculo ganancia sobre dataset de pruebas
test$score <- predicted
test$Predicted <- test$score > 0.025

#calculo la ganancia de incentivar cada caso
test[  , ganancia:= kFPGain ] #-1250
test[ clase==1,   ganancia:= kTPGain] #48750

#Determino ganancia
print(paste("La Ganancia Fold:",fold, 5*sum(test$Predicted*test$ganancia)))
ganancia_total <- ganancia_total + sum(test$Predicted*test$ganancia)   
    
}
print(paste("La Ganancia Total:",ganancia_total))

[1] "La Ganancia Fold: 1 7693750"
[1] "La Ganancia Fold: 2 8187500"
[1] "La Ganancia Fold: 3 7012500"
[1] "La Ganancia Fold: 4 8387500"
[1] "La Ganancia Fold: 5 7156250"
[1] "La Ganancia Total: 7687500"


In [22]:
#funcion que entrena/evalua ganancia para un foild
calculo_ganancia <-function(){
modelo   <-  rpart(formula,   data = train, maxdepth = 5,  cp=0,   xval=0)   

predicted <- predict(modelo, test,) # predicted values

#Calculo ganancia sobre dataset de pruebas
test$score <- predicted
test$Predicted <- test$score > 0.025

#calculo la ganancia de incentivar cada caso
test[  , ganancia:= kFPGain ] #-1250
test[ clase==1,   ganancia:= kTPGain] #48750
    
return(sum(test$Predicted*test$ganancia))
    }

In [29]:
#Numero de semilla que estoy probando
i <- 0

#Inicializo ganancias CV
ganancia_total_cv <- list()

#Inicializo ganancias Train/Test
ganancia_total_tt <- list()


for (seed in c(21023, 39761,20707,2049,7187,7207,9436519, 30689))
{
i <- i + 1

#Fija semilla    
set.seed(seed)

##### 5 Fold cross Validation ######
#Genero folds
folds <- createFolds(dataset$clase, 5, returnTrain = TRUE)
ganancia_total_cv[[i]] <- 0
for (fold in c(1,2,3,4,5))
{
#train_rows <- sample(1:nrow(dataset), .66 * nrow(dataset))
train <- dataset[unlist(folds[paste0('Fold',fold)]),]
test <- dataset[-unlist(folds[paste0('Fold',fold)]),]

#Determino ganancia
ganancia_total_cv[[i]]<- ganancia_total_cv[[i]] + calculo_ganancia()
  
}
#######

train_rows <- createDataPartition(dataset$clase, p = .66, list = FALSE)
ganancia_total_tt[[i]] <- 0
#train_rows <- sample(1:nrow(dataset), .66 * nrow(dataset))
train <- dataset[train_rows,]
test <- dataset[-train_rows,]
#Determino ganancia
ganancia_total_tt[[i]]<- ganancia_total_tt[[i]] + 3*calculo_ganancia()
  
}

In [30]:
ganancia_total_cv

[[1]]
[1] 6763750

[[2]]
[1] 6922500

[[3]]
[1] 7206250

[[4]]
[1] 7905000

[[5]]
[1] 7330000

[[6]]
[1] 7371250

[[7]]
[1] 6980000

[[8]]
[1] 7470000

In [31]:
ganancia_total_tt

[[1]]
[1] 8388750

[[2]]
[1] 12086250

[[3]]
[1] 7541250

[[4]]
[1] 9045000

[[5]]
[1] 6791250

[[6]]
[1] 6131250

[[7]]
[1] 6697500

[[8]]
[1] 6930000